# BL 1-5: SAXS data correction and reduction



#!/usr/bin/env python3

"""
Created on Fri Apr 21 23:41:02 2023

@author: akmaurya
"""

Global stuff

In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
#from axis_plot_prop import axis_plot_prop 
import pyFAI
import shutil
import fabio
from pyFAI.gui.jupyter.calib import Calibration

import os
import fnmatch


import warnings
warnings.filterwarnings('ignore')


# Common functions for the project

In [ ]:
# plot setting fucntion

def axis_plot_prop(ax):
    """
    Set axis properties for plotting.

    Parameters
    ----------
    ax : matplotlib axis object
        The axis object to set properties for.
    """

    co = [
        [0, 0, 0],
        [1, 0, 0],
        [0.44, 0.00, 0.99],
        [1.00, 0.50, 0.10],
        [0.75, 0.00, 0.75],
        [0.50, 0.50, 0.50],
        [0.50, 0.57, 0.00],
        [0.64, 0.08, 0.18],
        [0.93, 0.00, 0.00]
    ]

    plt.rcParams['axes.prop_cycle'] = plt.cycler(color=co)

    ax.set_facecolor('white')
    ax.spines['top'].set_linewidth(1)
    ax.spines['right'].set_linewidth(1)
    ax.spines['bottom'].set_linewidth(1)
    ax.spines['left'].set_linewidth(1)
    ax.tick_params(axis='both', which='both', direction='in', length=4, width=1)
    ax.minorticks_on()
    ax.set_xlabel('X Label', fontsize=20, fontname='Arial')
    ax.set_ylabel('Y Label', fontsize=20, fontname='Arial')
    ax.tick_params(axis='both', labelsize=20)
    ax.legend(frameon=False)
    #ax.set_box_aspect(1)
    plt.tight_layout()
#2D plot settings function
# import matplotlib.pyplot as plt

def set_pcolor_properties():
    """
    Sets the best figure properties for pcolormap plot
    """
    plt.rcParams["figure.figsize"] = (6, 6)  # Set the figure size to 8 inches by 6 inches
    plt.rcParams["font.size"] = 20  # Set the font size to 12 points
    plt.rcParams["axes.linewidth"] = 1.5  # Set the linewidth of the axes to 1.5
    #plt.rcParams["axes.grid"] = True  # Show the grid
    #plt.rcParams["grid.alpha"] = 0.5  # Set the transparency of the grid to 0.5
    #plt.rcParams["grid.linewidth"] = 0.5  # Set the linewidth of the grid to 0.5
    plt.rcParams["image.cmap"] = "viridis"  # Set the colormap to viridis
    plt.rcParams["image.interpolation"] = "nearest"  # Set the interpolation to nearest

# create and update folder to save data
def create_update_subfolder(base_folder, common_keyword):
    common_folder_path = os.path.join(base_folder, common_keyword)
    
    if os.path.exists(base_folder) and os.path.isdir(common_folder_path):
        print(f"Subfolder '{common_keyword}' already exists within the base folder.")
    else:
        os.makedirs(common_folder_path)
        print(f"Subfolder '{common_keyword}' created within the base folder.")

    return common_folder_path

# Convert pixel by pixel to qx by qy 2D SAXS plot

def convert_saxs_to_q(data, pixel_size, wavelength, detector_distance, beam_x, beam_y):
    # Define grid of pixel coordinates (assumes data is square)
    #x_pixels,y_pixels = data.shape
    x_pixels = 981
    y_pixels = 1043
    x_coords = np.arange(x_pixels) - beam_x
    y_coords = np.arange(y_pixels) - beam_y
    xx, yy = np.meshgrid(x_coords, y_coords)

    # Convert to q-space coordinates
    qx = 1e-9*2 * np.pi / wavelength * np.sin(pixel_size * xx / detector_distance)
    qy = 1e-9*2 * np.pi / wavelength * np.sin(pixel_size * yy / detector_distance)

    #print (len(xx))
    
    return qx, qy


    #print (qx)
    

def convert_waxs_to_q(data, pixel_size, wavelength, detector_distance, beam_x, beam_y):
    # Define grid of pixel coordinates (assumes data is square)
    #x_pixels,y_pixels = data.shape
    x_pixels = 487
    y_pixels = 195
    x_coords = np.arange(x_pixels) - beam_x
    y_coords = np.arange(y_pixels) - beam_y
    xx, yy = np.meshgrid(x_coords, y_coords)

    # Convert to q-space coordinates
    qx = 1e-9*2 * np.pi / wavelength * np.sin(pixel_size * xx / detector_distance)
    qy = 1e-9*2 * np.pi / wavelength * np.sin(pixel_size * yy / detector_distance)

    print (len(xx))
    
    return qx, qy


def set_plot_style(axs, fonts, xlabel, ylabel):
    axs.set_xlabel(xlabel, fontsize=fonts)
    axs.set_ylabel(ylabel, fontsize=fonts)
    axs.tick_params(axis='both', which='major', direction='out', length=4, width=1)
    axs.tick_params(which='minor', width=1, size=2)  # Adjust size as needed
    #axs.grid(True, which='both', axis='both', linestyle='--', linewidth=0.5)
    axs.set_facecolor('white')
    axs.spines['top'].set_linewidth(1)
    axs.spines['right'].set_linewidth(1)
    axs.spines['bottom'].set_linewidth(1)
    axs.spines['left'].set_linewidth(1)
    axs.tick_params(axis='x', labelsize=fonts)
    axs.tick_params(axis='y', labelsize=fonts)

    return axs    #print (qx) 
 


def darken_colors(num_colors, darker_factor=0.7):
    """
    Darken a set of colors by reducing the value component in the HSV color space.

    Parameters:
    - num_colors (int): Number of colors to generate.
    - darker_factor (float): Factor to control the darkness of the colors. Default is 0.7.

    Returns:
    - darker_colors (numpy.ndarray): Darkened colors in RGB format.
    """
    colors = plt.cm.jet(np.linspace(0, 1, num_colors))

    # Make the colors darker by reducing the value component
    darker_colors = colors.copy()

    for i in range(num_colors):
        rgb = darker_colors[i, :3]  # Extract the RGB values
        hsv = plt.cm.colors.rgb_to_hsv(rgb)  # Convert RGB to HSV
        hsv[2] *= darker_factor  # Reduce the value component
        darker_colors[i, :3] = plt.cm.colors.hsv_to_rgb(hsv)  # Convert back to RGB

    return darker_colors

# Inputs

# Poni

In [ ]:
#folder_path_poni_mask = '/Users/akmaurya/Library/CloudStorage/OneDrive-SLACNationalAcceleratorLaboratory/My Onedrive/Data_01/Methanolysis/Python/Data_Reduction/Convert_and_calibrate'
#folder_path_poni_mask = r'C:\Users\b_tassone\Desktop\Anjani\Python\Convert_and_calibrate'
#folder_path_poni_mask = '/Users/akmaurya/OneDrive - Stanford/Python_library/Acetolysis/May2024/Python/Convert_and_calibrate/Convert_and_calibrate'

base_dir = r'/Users/alexisvoulgaropoulos/Library/CloudStorage/OneDrive-Stanford/BOTTLE/BEAMTIME/May2025_BL1-5_data/20250501/'
folder_path_poni_mask = os.path.join(base_dir, 'poni')


SAXS_poni_file = "SAXS_AgBeh_solvent_02.poni" #loading the AgBeh standard
WAXS_poni_file = "SAXS_AgBeh_solvent_02.poni" #loading the AgBeh standard

# SAXS

##### I think this remains the same for the toluene runs... double check though (- June 2, 2025 -)

In [ ]:
poni_file_path = os.path.join(folder_path_poni_mask,SAXS_poni_file)
print(poni_file_path)
ai = pyFAI.load(poni_file_path)

print(ai)

#Define detector parameters for 2D q conversion
pixel_size = ai.get_pixel1() # in meters 
wavelength = ai.get_wavelength() #1.54e-10  # in meters
detector_distance = ai.get_dist()  # in meters
beam_x=488.787 #from May 2025 Igor file during beamtime
beam_y=838.494 # beam center x-coordinate in pixels


#mask for SAXS - June 2, 2025 - need to actually make one

In [ ]:
# Load the poni file and extract relevant parameters
saxs_mask = os.path.join(folder_path_poni_mask, "SAXS_solvent_mask_03.edf")

#mask = "/Users/yuewu/Desktop/8091E/BL15_1M_gcarbon_mask_3.edf"
saxs_mask = fabio.open(saxs_mask).data
plt.imshow(saxs_mask)# display the mask

# WAXS

In [ ]:

# Load the poni file and extract relevant parameters
poni_file_path = os.path.join(folder_path_poni_mask, WAXS_poni_file)
ai_w = pyFAI.load(poni_file_path)

print(ai_w)



#Define detector parameters for 2D q conversion
pixel_size = ai_w.get_pixel1() # in meters 
wavelength = ai_w.get_wavelength() #1.54e-10  # in meters
detector_distance = ai_w.get_dist()  # in meters
beam_x= 668.647
beam_y= 105.765 # beam center x-coordinate in pixels


#mask for WAXS

In [ ]:
# Load the poni file and extract relevant parameters
waxs_mask = os.path.join(folder_path_poni_mask, "SAXS_solvent_mask_03.edf")

#mask = "/Users/yuewu/Desktop/8091E/BL15_1M_gcarbon_mask_3.edf"
waxs_mask = fabio.open(waxs_mask).data
plt.imshow(waxs_mask)# display the mask

# offsets and air

In [ ]:
# get this from Empty Beam: shutter closed 
# is this from one of the first samples?

i0_offset = 0
bstop_offset = 0

# average from without sample in the beam: named as "Air" is the standard 

#May2025: From macro72_20250511_air_pretTolRun3
i0_air = 43.138718
bstop_air = 45.938285



# 

## function to calculate mu and thickness

In [ ]:
import periodictable as pt
import xraydb
import numpy as np

def calculate_sld_mu_thickness(energy_keV, density, transmission):
    try:
        # Define PE composition correctly
        material = pt.formula("C7H8")  # Changing this to C7H8 for toluene

        # Calculate scattering length density (SLD) and mu using periodictable
        sld, mu_pt = pt.xray_sld(material, energy=energy_keV, density=density)

        # Use xraydb for a more reliable absorption coefficient (mu) calculation
        mu_xraydb = xraydb.material_mu("C2H4", energy=energy_keV * 1000, density=density) * 100  # Convert to 1/m

        # Calculate thickness using transmission: T = exp(-mu * t) → t = -ln(T) / mu
        if transmission <= 0 or transmission > 10:
            raise ValueError("Transmission must be in the range (0, 1].")
        
        thickness = -np.log(transmission) / mu_xraydb  # Thickness in meters

        return {"mu": mu_xraydb, "sld": sld, "thickness_m": thickness}

    except Exception as e:
        raise ValueError(f"Error in calculating SLD, mu, and thickness for PE: {e}")

# Example usage
energy_keV = 15      # Energy in keV
density_PE = 0.867       # Density in g/cm^3
#transmission = 0.92   # Transmission (must be between 0 and 1)

#result = calculate_sld_mu_thickness(energy_keV, density_PE, transmission)
#print(result)


# Set the directory containing the raw and pdi files and list the keyword

In [ ]:
import os
import fnmatch

def find_folders_with_keyword(folder_path, keyword, skip_folders=None):
    folder_names = []
    for item in os.listdir(folder_path):
        item_path = os.path.join(folder_path, item)
        if os.path.isdir(item_path) and fnmatch.fnmatch(item, keyword):
            if skip_folders is None or item not in skip_folders:
                folder_names.append(item)

    return folder_names

def get_keywords(raw_dir, common_keyword, keyword):
    data_folder = os.path.join(raw_dir, common_keyword)
    base_SAXS_folder = "OneD_integrated_SAXS_01/Reduction"
    base_SAXS_folder = os.path.join(raw_dir, base_SAXS_folder)
    base_WAXS_folder = "OneD_integrated_WAXS_01/Reduction"
    base_WAXS_folder = os.path.join(raw_dir, base_WAXS_folder)

    SAXS_folder_name = create_update_subfolder(base_SAXS_folder, common_keyword)
    WAXS_folder_name = create_update_subfolder(base_WAXS_folder, common_keyword)

    folder_names_with_keyword = find_folders_with_keyword(data_folder, keyword)

    #ctr_values = [int(name.split('_ctr')[-1]) for name in folder_names_with_keyword]
    sorted_folder_names = sorted(folder_names_with_keyword, key=custom_sort)

    keywords = sorted_folder_names
    return keywords, SAXS_folder_name, WAXS_folder_name,data_folder




# read parameters function

In [ ]:
def process_and_store_selected_parameters_columns(data_folder, keywords):
    # Create an empty dictionary to store the selected columns for each file
    selected_columns_dict = {}

    for keyword in keywords:
        # Create a list of file paths that match the file types and current keyword
        #raw_file_pattern = f'*{keyword}/SAXS/*.raw'
        pdi_file_pattern = f'*{keyword}/*_scan1.csv'
        #raw_file_paths = sorted(glob.glob(os.path.join(raw_dir, raw_file_pattern)))
        pdi_file_paths = sorted(glob.glob(os.path.join(data_folder, pdi_file_pattern)))

        for pdi_file_path in pdi_file_paths:
            try:
                # Read the CSV file
                df = pd.read_csv(pdi_file_path)
                file_name = os.path.splitext(os.path.basename(pdi_file_path))[0]
                #print(f"Reading file: {file_name}")

                # Extract columns 4, 7, 9, and 15
                selected_columns = df.iloc[:, [2, 3, 6, 10, 29, 30]]

                # Store the selected columns in the dictionary with the file name as the key
                selected_columns_dict[file_name] = selected_columns

            except FileNotFoundError:
                print(f"File not found: {pdi_file_path}")
            except Exception as e:
                print(f"An error occurred: {e}")

    return selected_columns_dict

# Call the function and pass the raw_dir and keywords as arguments
#raw_dir = 'path_to_your_raw_files_directory'
#keywords = ['keyword1', 'keyword2', 'keyword3']  # Replace with your actual keywords
#
# To load the result_dict back from the file, you can use the following code:

#print(result_dict)


In [ ]:
import pandas as pd
import os
import glob

def read_parameters(data_folder, keyword, result_dict):
    """
    Reads and processes parameter data from CSV files matching a pattern, calculates averages, 
    and returns a dictionary with results.

    Args:
        data_folder (str): Path to the folder containing CSV files.
        keyword (str): Keyword to filter files.
        result_dict (dict): Dictionary containing preloaded DataFrames indexed by filenames.

    Returns:
        dict: Dictionary containing averages and filenames.
    """
    # Initialize lists to store results
    bstop_dx_avg, ctemp_dx_avg, timer_dx_avg, i0_dx_avg = [], [], [], []
    bstop_avg_all, ctemp_avg_all, timer_avg_all, i0_avg_all = [], [], [], []
    avg_file_name = []

    # File matching pattern
    pdi_file_pattern = f'*{keyword}/*_scan1.csv'
    pdi_file_paths = sorted(glob.glob(os.path.join(data_folder, pdi_file_pattern)))

    for pdi_file_path in pdi_file_paths:
        file_name = os.path.splitext(os.path.basename(pdi_file_path))[0]
        selected_columns = result_dict.get(file_name)

        if selected_columns is not None:
            try:
                # Access columns safely
                i0 = pd.to_numeric(selected_columns.iloc[:, 1], errors='coerce')
                bstop = pd.to_numeric(selected_columns.iloc[:, 2], errors='coerce')
                ctemp = pd.to_numeric(selected_columns.iloc[:, 4], errors='coerce')
                timer = pd.to_numeric(selected_columns.iloc[:, 5], errors='coerce')

                # Calculate averages
                i0_dx_avg.append(i0.mean(skipna=True))
                bstop_dx_avg.append(bstop.mean(skipna=True))
                ctemp_dx_avg.append(ctemp.mean(skipna=True))
                timer_dx_avg.append(timer.mean(skipna=True))

                # Add filename to results
                avg_file_name.append(file_name)
            except (ValueError, IndexError) as e:
                print(f"Error processing file '{file_name}': {str(e)}")
        else:
            print(f"File '{file_name}' not found in result_dict.")

    # Compute overall averages if data exists
    if i0_dx_avg:
        i0_avg_all.append(sum(i0_dx_avg) / len(i0_dx_avg))
        bstop_avg_all.append(sum(bstop_dx_avg) / len(bstop_dx_avg))
        ctemp_avg_all.append(sum(ctemp_dx_avg) / len(ctemp_dx_avg))
        timer_avg_all.append(sum(timer_dx_avg) / len(timer_dx_avg))

    return {
        'bstop_dx_avg': bstop_dx_avg,
        'ctemp_dx_avg': ctemp_dx_avg,
        'timer_dx_avg': timer_dx_avg,
        'i0_dx_avg': i0_dx_avg,
        'bstop_avg_all': bstop_avg_all,
        'ctemp_avg_all': ctemp_avg_all,
        'timer_avg_all': timer_avg_all,
        'i0_avg_all': i0_avg_all,
        'avg_file_name': avg_file_name,
    }


# SAXS Data reduction function

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import glob
import re
#import warnings

# Filter warnings containing the substring "pyFAI.io"
#warnings.filterwarnings("ignore", message="pyFAI.io")

# Map a parameter (e.g., q) to a colormap


import re

def custom_sort(file_name):
    keywords = ['', '', 'HoldT', 'Cooling', 'RampT', '']
    
    # Find the first matching keyword in file_name
    keyword_found = next((kw for kw in keywords if kw in file_name), '')
    
    # Extract ctr value using regex
    ctr_match = re.search(r'ctr(\d+)', file_name)
    ctr_value = int(ctr_match.group(1)) if ctr_match else 0  # Default to 0 if not found

    return (keywords.index(keyword_found), ctr_value)


def process_SAXS_data(SAXS_folder_name, keywords, data_folder, ai, mask, i0_offset, bstop_offset, i0_air, bstop_air):
    # print the processting saxs data
    print("Processing SAXS data...")

    save_SAXS_norm_files = os.path.join(SAXS_folder_name.replace("Reduction", ""), "Correction/Normlised")
    if not os.path.exists(save_SAXS_norm_files):
        os.makedirs(save_SAXS_norm_files)
    result_dict = process_and_store_selected_parameters_columns(data_folder, keywords)
    #warnings.filterwarnings('ignore')
    bstop = []
    ctemp = []
    timer = []
    i0 = []
    filename_p = []
    file_name_norm = []



    # Create a figure with 1 row and 2 columns
    fig, axs = plt.subplots(figsize=(8,8))
    c=0
    
    num_colors = len(keywords)  # Calculate the number of colors based on the maximum m_key
    
    print(num_colors)
    darker_colors = darken_colors(num_colors, darker_factor=0.7)
    for keyword in keywords:
        # Create a list of file paths that match the file types and current keyword
        raw_file_pattern = f'{keyword}/SAXS/*.raw'
        #raw_file_paths = sorted(glob.glob(os.path.join(data_folder, raw_file_pattern)))
        raw_file_paths = sorted(glob.glob(os.path.join(data_folder, raw_file_pattern)), key=custom_sort)
        #print(raw_file_paths)
        #print(keyword)


        # read parameters
        # Call the function
        averages_dict = read_parameters(data_folder, keyword, result_dict)

        # Access the averages from the returned dictionary
        bstop_avg_all = averages_dict['bstop_avg_all']
        ctemp_avg_all = averages_dict['ctemp_avg_all']
        timer_avg_all = averages_dict['timer_avg_all']
        i0_avg_all = averages_dict['i0_avg_all']
        avg_file_name = averages_dict['avg_file_name']


        #print(avg_file_name, raw_file_pattern, bstop_avg_all, ctemp_avg_all, timer_avg_all, i0_avg_all)

        bstop.append(bstop_avg_all[0])
        ctemp.append(ctemp_avg_all[0])
        timer.append(timer_avg_all[0])
        i0.append(i0_avg_all[0])
        filename_p.append(avg_file_name[0])  

        #print(i0_offset, bstop_offset)

        i0_avg_all[0] = i0_avg_all[0] - i0_offset
        bstop_avg_all[0] = bstop_avg_all[0] - bstop_offset

        i0_air = i0_air - i0_offset
        bstop_air = bstop_air - bstop_offset

        trans_factor = (bstop_avg_all[0])#/i0_avg_all[0])/(bstop_air/i0_air)
        #trans_factor_t = (bstop_avg_all[0])/i0_avg_all[0]
        #print(trans_factor)
        normfactor = trans_factor*i0_avg_all[0]

        #result = calculate_sld_mu_thickness(energy_keV, density_PE, trans_factor_t)
        #thickness = result['thickness_m']
        #thickness = 0.0001

        #normfactor = trans_factor #* thickness
        normfactor = float(normfactor)
        #print the transfactor, thicknes and normfactor in good format
        print(f"i0:{i0_avg_all[0]}, bstop: {bstop_avg_all[0]}, normfactor: {normfactor}")

        #normfactor = 1.0
        #print(normfactor)

        # Initialize the accumulators for the averaged image and pdi data
        avg_image = np.zeros((1043, 981))

        for i, raw_file_path in enumerate(raw_file_paths):
            # Read the raw file
            data = np.fromfile(raw_file_path, dtype=np.int32).reshape(1043, 981)
            avg_image += data

        avg_image /= len(raw_file_paths)

        # Assuming raw_file_paths[-1] contains the file path
        file_name, file_extension = os.path.splitext(os.path.basename(raw_file_paths[-1]))
        file_name = file_name.split('_scan')[0]
        file_name = file_name.split('sone_')[1]

        # Remove the last 10 characters from the file name and the ".raw" extension
        #file_name = file_name[5:-11]
        file_name=f"{file_name}_all_SAXS.dat"
        file_name_norm.append(file_name)

        q, I, error_avg = ai.integrate1d(avg_image, 1000, error_model='poisson', mask=mask, normalization_factor=normfactor, filename=os.path.join(SAXS_folder_name, file_name))
        
        I_norm = I
        I_norm_sigma = error_avg
        
        axs.loglog(q, I, label=f"{file_name}", color=darker_colors[c])
        axs.set_xlabel("q (nm$^{-1}$)")
        axs.set_ylabel("Intensity (a.u.)")
        #axs.set_xlim([0.05, 3])
        set_plot_style(axs,20, 'q (nm$^{-1}$)', 'Intensity (a.u.)')
        #axs.legend(fontsize=12)




        # Create a DataFrame for subtracted data
        data = {"q_nm^-1": q, "I_avg_nomrlised": I_norm, "I_norm_sigma": I_norm_sigma}
        df = pd.DataFrame(data)

        # Saving the normalized data to a file
        # remove .dat extension
        file_name = file_name[:-4]

        # remove a string from middle of the file name from _10s to _d and attached first and last part




        dat_filename = f"{save_SAXS_norm_files}/{file_name}_Norm.dat"

        inst_parameters = f'Timer: {timer_avg_all[0]}, bstop: {bstop_avg_all[0]}, ctemp: {ctemp_avg_all[0]}, I0: {i0_avg_all[0]}'
        headers = [
            f"filename: {file_name}",
            f"background : {''}",
            f'Empty : {""}',
            f"{inst_parameters}", 
            'fit_data',
            "q_nm^-1 ------ I_avg_nomrlised ------ I_norm_sigma"]
        commented_headers = ['# ' + header for header in headers]

        with open(dat_filename, 'w') as dat_file:
            dat_file.write('\n'.join(commented_headers) + '\n')
            df.to_csv(dat_file, sep='\t', index=False, header=False)


        plt.tight_layout()
        
        c=c+1



    plt.show()
    

    # Initialize the result list with the first value as 0
    duration = [0]

    # Extracting the minimum timestamp from all the timer values
    min_timer = min(timer)
    #print(timer)

    # Subtract the minimum timestamp from each timer value and append to the result list
    for i in range(1, len(timer)):
        duration_avg = (timer[i] - min_timer) / 3600
        duration.append(duration_avg)
    
    # save the filename, duration, bstop, ctemp, i0 values in a xlsx file
   
    df = pd.DataFrame(list(zip(file_name_norm, timer, bstop, ctemp, i0)), columns =['Filename', 'Timer', 'Bstop', 'Ctemp', 'I0'])
    df.to_excel(os.path.join(SAXS_folder_name, 'SAXS_parameters_M12.xlsx'), index=False)




# Create a figure and subplots
    fig, ((axs1,axs2), (axs3, axs4), (axs5, axs6)) = plt.subplots(3,2, figsize=(10, 12))


    # Plot data on the first subplot
    axs1.plot(duration, bstop, 'o', color='b', label='bstop')
    #axs1b = axs1.twinx()
    axs1.plot(duration, i0, 'o', color='r', label='I0')
    #axs1.grid(True)
    axs1.legend()
    #axs1b.legend()
    axs1.set_xlabel('Duration [hr]')
    axs1.set_ylabel('Bstop and I0')
    #axs1b.set_ylabel('I0')
    set_plot_style(axs1,20, 'Duration', 'Bstop')
    #set_plot_style(axs1b,20, 'Duration', 'I0')
    

    # Plot data on the second subplot
    axs2.plot(duration, ctemp, 'o', color='r', label='ctemp')
    #axs2.grid(True)
    axs2.legend()
    axs2.set_xlabel('Duration')
    axs2.set_ylabel('Ctemp')
    set_plot_style(axs2,20, 'Duration [hr]', 'Ctemp [C]')

    #Creating subplots with just bstop and i0 plots

    axs3.plot(duration, i0, 'o', color='r', label='I0')
    axs3.legend()
    axs3.set_xlabel('Duration [hr]')
    axs3.set_ylabel('I0')
    set_plot_style(axs3,15, 'Duration [hr]', 'I0')

    axs4.plot(duration, bstop, 'o', color='b', label='bstop')
    axs4.legend()
    axs4.set_xlabel('Duration [hr]')
    axs4.set_ylabel('Bstop')
    set_plot_style(axs4,15, 'Duration [hr]', 'Bstop')

    #Double y axis plot for duration, bstop, and i0
    line5 = axs5.plot(duration, bstop, 'o', color='b', label='bstop')
    axs5b = axs5.twinx()
    line5b = axs5b.plot(duration, i0, 'o', color='r', label='I0')
    lines5 = line5 + line5b
    labs5 = [l.get_label() for l in lines5]
    axs5.legend(lines5, labs5, loc=0)
    axs5.set_xlabel('Duration [hr]')
    axs5.set_ylabel('Bstop')
    axs5b.set_ylabel('I0')
    set_plot_style(axs5,15, 'Duration', 'bstop')
    set_plot_style(axs5b,15, 'Duration', 'I0')

    #Double y axis plot for temperature, bstop, and i0
    line6 = axs6.plot(ctemp, bstop, 'o', color='b', label='bstop')
    axs6b = axs6.twinx()
    line6b = axs6b.plot(ctemp, i0, 'o', color='r', label='I0')
    lines6 = line6 + line6b
    labs6 = [l.get_label() for l in lines6]
    axs6.legend(lines6, labs6, loc=0)
    axs6.set_xlabel('Temperature [C]')
    axs6.set_ylabel('Bstop')
    axs6b.set_ylabel('I0')
    set_plot_style(axs6,15, 'Temp', 'Bstop')
    set_plot_style(axs6b,15, 'Temp', 'I0')
    




    # Show the plots
    plt.tight_layout()
    plt.show()





'''
#raw_dir_aceto = '/Users/akmaurya/OneDrive - Stanford/Data_01/Acetolysis/May2024/'
#raw_dir_meth = 'C:/Users/akmaurya/OneDrive - Stanford/Data_01/Methanolysis/May2024/'

raw_dir = r'X:\bl1-5\Anjani\Autoxidation\Jan2025\atT'


# Run1
common_keyword = ""
keyword = "Run5*"

keywords, SAXS_folder_name, WAXS_folder_name,data_folder = get_keywords(raw_dir, common_keyword, keyword)
"""
print("Keywords:")
for kw in keywords:
    print(f"    '{kw}',")
"""

#process_SAXS_data(SAXS_folder_name, keywords, data_folder, ai)
#process_WAXS_data(WAXS_folder_name, keywords, data_folder, ai_w)
'''


# WAXS Data reduction function

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import re

import warnings

# Filter warnings containing the substring "pyFAI.io"
warnings.filterwarnings("ignore", message="pyFAI.io")

# Your plotting code here...
def custom_sort(file_name):
    keywords = ['ctr', 'RampT', 'HoldT', 'Cooling', '']
    keyword_found = next((kw for kw in keywords if kw in file_name), '')
    ctr_value = int(file_name.split('ctr')[-1].split('_')[0]) if 'ctr' in file_name else 0
    return (keywords.index(keyword_found), ctr_value)

def process_WAXS_data(WAXS_folder_name, keywords, data_folder, ai_w, mask, i0_offset, bstop_offset, i0_air, bstop_air):
    # print the processting waxs data
    print("Processing WAXS data...")
    save_SAXS_norm_files = os.path.join(WAXS_folder_name.replace("Reduction", ""), "Correction/Normlised")
    if not os.path.exists(save_SAXS_norm_files):
        os.makedirs(save_SAXS_norm_files)

    result_dict = process_and_store_selected_parameters_columns(data_folder, keywords)
    #warnings.filterwarnings('ignore')
            # Create a figure with 1 row and 2 columns
    fig, axs = plt.subplots(figsize=(12,12))
    
    bstop = []
    ctemp = []
    timer = []
    i0 = []
    filename_p = []
    file_name_norm = []

    c=0 
    
    num_colors = len(keywords)
    
    darker_colors = darken_colors(num_colors, darker_factor=0.7)

    for keyword in keywords:
        # Create a list of file paths that match the file types and current keyword
        raw_file_pattern = f'{keyword}/WAXS/*.raw'
        #raw_file_paths = sorted(glob.glob(os.path.join(data_folder, raw_file_pattern)))
        raw_file_paths = sorted(glob.glob(os.path.join(data_folder, raw_file_pattern)), key=custom_sort)


# Call the function

        averages_dict = read_parameters(data_folder, keyword, result_dict)

        # Access the averages from the returned dictionary
        bstop_avg_all = averages_dict['bstop_avg_all']
        ctemp_avg_all = averages_dict['ctemp_avg_all']
        timer_avg_all = averages_dict['timer_avg_all']
        i0_avg_all = averages_dict['i0_avg_all']
        avg_file_name = averages_dict['avg_file_name']

        

        #print(avg_file_name, raw_file_pattern, bstop_avg_all, ctemp_avg_all, timer_avg_all, i0_avg_all)

        bstop.append(bstop_avg_all[0])
        ctemp.append(ctemp_avg_all[0])
        timer.append(timer_avg_all[0])
        i0.append(i0_avg_all[0])
        filename_p.append(avg_file_name[0]) 

        i0_avg_all[0] = i0_avg_all[0] - i0_offset
        bstop_avg_all[0] = bstop_avg_all[0] - bstop_offset

        i0_air = i0_air - i0_offset
        bstop_air = bstop_air - bstop_offset

        trans_factor = (bstop_avg_all[0])#/i0_avg_all[0])/(bstop_air/i0_air)
        trans_factor_t = (bstop_avg_all[0])/i0_avg_all[0]
        #print(trans_factor)
        trans_factor = trans_factor*i0_avg_all[0]

        result = calculate_sld_mu_thickness(energy_keV, density_PE, trans_factor_t)
        thickness = result['thickness_m']

        normfactor = trans_factor #* thickness
        normfactor = float(normfactor)
        #print the transfactor, thicknes and normfactor in good format
        print(f"i0:{i0_avg_all[0]}, bstop: {bstop_avg_all[0]}, transmission factor: {trans_factor_t}, thickness: {thickness}, normfactor: {normfactor}")

        normfactor = float(normfactor)
        #normfactor = 1.0
        #print(normfactor)


        # Initialize the accumulators for the averaged image and pdi data
        avg_image = np.zeros((195, 487))

        for i, raw_file_path in enumerate(raw_file_paths):
            # Read the raw file
            data = np.fromfile(raw_file_path, dtype=np.int32).reshape(195, 487)
            avg_image += data

            file_name = os.path.splitext(os.path.basename(raw_file_path))[0]

        avg_image /= len(raw_file_paths)

        # Assuming raw_file_paths[-1] contains the file path
        file_name, file_extension = os.path.splitext(os.path.basename(raw_file_paths[-1]))

        # Remove the last 10 characters from the file name and the ".raw" extension
        # split the file name _scan and remove the last part of the file name
        file_name = file_name.split('_scan')[0]
        file_name=f"{file_name}_all_WAXS.dat"
        file_name=file_name.split('b_tassone_')[1]
        file_name_norm.append(file_name)



        q, I, error_avg = ai_w.integrate1d(avg_image, 1000,error_model='poisson', mask=mask,normalization_factor=normfactor,filename=os.path.join(WAXS_folder_name, file_name))
   
        I_norm = I
        I_norm_sigma = error_avg

        axs.plot(q, I, '-', label=f"{file_name}",color=darker_colors[c])
        axs.set_xlabel("q (nm$^{-1}$)")
        axs.set_ylabel("Intensity (a.u.)")
        set_plot_style(axs,20, 'q (nm$^{-1}$)', 'Intensity (a.u.)')
        #axs.set_xlim([0.05, 3])
        #axs.legend(fontsize=12)

        # Create a DataFrame for subtracted data
        data = {"q_nm^-1": q, "I_avg_nomrlised": I_norm, "I_norm_sigma": I_norm_sigma}
        df = pd.DataFrame(data)

        # Saving the normalized data to a file
        # remove .dat extension
        file_name = file_name[:-4]
        dat_filename = f"{save_SAXS_norm_files}/{file_name}_Norm.dat"

        inst_parameters = f'Timer: {timer_avg_all[0]}, bstop: {bstop_avg_all[0]}, ctemp: {ctemp_avg_all[0]}, I0: {i0_avg_all[0]}'
        headers = [
            f"filename: {file_name}",
            f"background : {''}",
            f'Empty : {""}',
            f"{inst_parameters}", 
            'fit_data',
            "q_nm^-1 ------ I_avg_nomrlised ------ I_norm_sigma"]
        commented_headers = ['# ' + header for header in headers]

        with open(dat_filename, 'w') as dat_file:
            dat_file.write('\n'.join(commented_headers) + '\n')
            df.to_csv(dat_file, sep='\t', index=False, header=False)



        c = c+1
        plt.tight_layout()
        
        
    plt.show()


    # Initialize the result list with the first value as 0
    duration = [0]

    # Extracting the minimum timestamp from all the timer values
    min_timer = min(timer)
    #print(timer)

    # Subtract the minimum timestamp from each timer value and append to the result list
    for i in range(1, len(timer)):
        duration_avg = (timer[i] - min_timer) / 3600
        duration.append(duration_avg)
    # save the filename, duration, bstop, ctemp, i0 values in a xlsx file
   
    df = pd.DataFrame(list(zip(file_name_norm, timer, bstop, ctemp, i0)), columns =['Filename', 'Timer', 'Bstop', 'Ctemp', 'I0'])
    df.to_excel(os.path.join(WAXS_folder_name, 'WAXS_parameters.xlsx'), index=False)



# Create a figure and subplots
    fig, (axs1,axs2) = plt.subplots(1,2, figsize=(10, 4))

    # Plot data on the first subplot
    axs1.plot(duration, bstop, 'o', color='b', label='bstop')
    axs1.plot(duration, i0, 'o', color='r', label='I0')
    #axs1.grid(True)
    axs1.legend()
    axs1.set_xlabel('Duration [hr]')
    axs1.set_ylabel('Bstop and I0')
    set_plot_style(axs1,20, 'Duration', 'Bstop, I0')
    

    # Plot data on the second subplot
    axs2.plot(duration, ctemp, 'o', color='r', label='ctemp')
    #axs2.grid(True)
    axs2.legend()
    axs2.set_xlabel('Duration')
    axs2.set_ylabel('Ctemp')
    set_plot_style(axs2,20, 'Duration [hr]', 'Ctemp [C]')

    # Show the plots
    plt.tight_layout()
    plt.show()




# Ploting function

In [ ]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import numpy as np
import matplotlib.pyplot as plt

def darken_colors(num_colors, darker_factor=0.7):
    """
    Darken a set of colors by reducing the value component in the HSV color space.

    Parameters:
    - num_colors (int): Number of colors to generate.
    - darker_factor (float): Factor to control the darkness of the colors. Default is 0.7.

    Returns:
    - darker_colors (numpy.ndarray): Darkened colors in RGB format.
    """
    colors = plt.cm.jet(np.linspace(0, 1, num_colors))

    # Make the colors darker by reducing the value component
    darker_colors = colors.copy()

    for i in range(num_colors):
        rgb = darker_colors[i, :3]  # Extract the RGB values
        hsv = plt.cm.colors.rgb_to_hsv(rgb)  # Convert RGB to HSV
        hsv[2] *= darker_factor  # Reduce the value component
        darker_colors[i, :3] = plt.cm.colors.hsv_to_rgb(hsv)  # Convert back to RGB

    return darker_colors

def read_data_files(folder_path, keywords):
    data_list = []
    m_key = 0  # Initialize m_key to 0
    m_keys = []  # Create a list to store m_key for each file
    legend = []  # Create a list to store legends

    def sort_key(filename):
        ctr_index = filename.find("ctr")
        if ctr_index != -1:
            numeric_part = filename[ctr_index + 3:]
            numeric_part = numeric_part.split('_')[0]  # Extract numeric part before the next underscore
            try:
                return int(numeric_part)
            except ValueError:
                return float('inf')
        else:
            return float('inf')

    for keyword in keywords:
        files = glob.glob(os.path.join(folder_path, f"*{keyword}*.dat"))
        files = sorted(files, key=lambda x: (sort_key(x), x))

        for file in files:
            try:
                with open(file, 'r') as f:
                    lines = f.readlines()

                start_index = None
                for i, line in enumerate(lines):
                    if line.strip().startswith("#"):
                        continue
                    else:
                        start_index = i
                        break

                if start_index is not None:
                    df = pd.read_csv(file, delim_whitespace=True, skiprows=start_index, header=None,
                                     names=["q_nm^-1", "I", 'sigma', ])
                    df["File"] = file
                    if not df.empty:
                        data_list.append(df)

                        # Extract temperature from the filename
                        # temperature = file.split('_T')[1].split('_')[0]

                        # Print only the file name (without path) just before plotting
                        file_name = os.path.basename(file)
                        #print(f" {file_name}")

                        # Check if temperature is present in the filename

                        legend.append(file_name)

                        m_keys.append(m_key)  # Append m_key for this file to the list
            except Exception as e:
                print(f"Error reading file {file}: {str(e)}")
            m_key += 1  # Increment m_key for each file
        m_key += 1  # Increment m_key by 2 when switching keywords

    # After the loop, concatenate the DataFrames
    data = pd.concat(data_list, ignore_index=True)

    # print(legend)
    return data, m_keys, legend



import matplotlib.pyplot as plt

def create_saxs_waxs_plot(folder_path_saxs, folder_path_waxs, keywords, m_keys, data_saxs, data_waxs, legend_saxs, legend_waxs):
    # Create a 1x2 grid of subplots
    fig = plt.figure(figsize=(12, 12))
    fonts = 20
    
    num_colors = max(m_keys) + 1  # Calculate the number of colors based on the maximum m_key
    darker_colors = darken_colors(num_colors, darker_factor=0.7)
    
    #print(keywords)
    
    # SAXS plot
    plt.subplot(1, 2, 1)
    plt.title('SAXS',fontsize=fonts)
    for i, file in enumerate(data_saxs["File"].unique()):
        subset = data_saxs[data_saxs["File"] == file]
        qmin, qmax = 0.07, 0.8
        subset = subset[(subset["q_nm^-1"] >= qmin) & (subset["q_nm^-1"] <= qmax)]
        
        # Check if any data points are present before plotting
        if not subset.empty:
            #print(file)
            #print(subset)
            plt.loglog(subset["q_nm^-1"], subset["I"] * (1 ** m_keys[i]), markersize=0.5, color=darker_colors[m_keys[i]],label=legend_saxs[i].replace('_all_SAXS.dat', '') if legend_saxs is not None else None)

    plt.xlim(0.07, 0.8)
    plt.xlabel('q [$\\mathrm{nm^{-1}}$]', fontsize=fonts)
    plt.ylabel("Intensity [a.u.]", fontsize=fonts)
    plt.xticks(fontsize=fonts)  # Set x-axis tick to 20
    plt.yticks(fontsize=fonts)  # Set y-axis tick to 20
    
    plt.gca().set_facecolor('white')
    plt.gca().spines['top'].set_linewidth(1)
    plt.gca().spines['right'].set_linewidth(1)
    plt.gca().spines['bottom'].set_linewidth(1)
    plt.gca().spines['left'].set_linewidth(1)
    plt.tick_params(axis='both', which='both', direction='out', length=4, width=1)

    #plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=10, frameon=False)
    #plt.legend(loc='upper right', fontsize=10, frameon=True)

    # WAXS plot
    plt.subplot(1, 2, 2)
    plt.title('WAXS',fontsize=fonts)
    for i, file in enumerate(data_waxs["File"].unique()):
        subset = data_waxs[data_waxs["File"] == file]
        
        qmin, qmax = 10, 40
        subset = subset[(subset["q_nm^-1"] >= qmin) & (subset["q_nm^-1"] <= qmax)]
        
        
        plt.plot(subset["q_nm^-1"], subset["I"] + (0* m_keys[i]), markersize=0.5,color=darker_colors[m_keys[i]], label=legend_waxs[i].replace('_all_WAXS.dat', '') if legend_waxs is not None else None)

    #plt.xlim(10, 25)
    plt.xlabel('q [$\\mathrm{nm^{-1}}$]', fontsize=fonts)
    plt.ylabel("Intensity [a.u.]", fontsize=fonts)
    plt.xticks(fontsize=fonts)  # Set x-axis tick to 20
    plt.yticks(fontsize=fonts)  # Set y-axis tick to 20
    plt.gca().set_facecolor('white')
    plt.gca().spines['top'].set_linewidth(1)
    plt.gca().spines['right'].set_linewidth(1)
    plt.gca().spines['bottom'].set_linewidth(1)
    plt.gca().spines['left'].set_linewidth(1)
    plt.tick_params(axis='both', which='both', direction='out', length=4, width=1)

    #plt.legend(loc='center left', bbox_to_anchor=(1.1, 0.5), fontsize=8, frameon=False)

    

    # Adjust layout and show the plot
    plt.tight_layout()
    
    
    
    ###################### save figure
    # Use the first 30 characters of the first keyword as the figure name
    figure_name = keywords[0][:30].replace('*', '')
    
    
    # Save the figure to the specified folder path with the given name
    #plt.savefig(f"{folder_path}/{figure_name}.png", dpi=300)
    
    
    # Create the "Figure" folder if it doesn't exist
    #figure_folder = os.path.join(folder_path_saxs, 'Figure_SAXS_WAXS')
    #os.makedirs(figure_folder, exist_ok=True)
    figure_folder = os.path.join(os.path.dirname(folder_path_saxs), 'Figure_saxs_waxs_Reduced')
    os.makedirs(figure_folder, exist_ok=True)


    # Save the figure in the specified folder
    figure_path = os.path.join(figure_folder, f'{figure_name}_SAXS_WAXS.png')
    plt.savefig(figure_path, dpi=300)

    ###################
    plt.show()
    

        
        

        
def plot_saxs_waxs_data(all_keywords, Run_number):
    #folder_path_saxs_base = r'X:\bl1-5\Anjani\Autoxidation\Jan2025\RT\RTSoak\OneD_integrated_SAXS_01\Reduction'
    #folder_path_waxs_base = r'X:\bl1-5\Anjani\Autoxidation\Jan2025\RT\RTSoak\OneD_integrated_WAXS_01\Reduction'
    folder_path_saxs_base = r'X:\bl1-5\Alexis\2025\202505\20250501\OneD_integrated_SAXS_01\Reduction'
    folder_path_waxs_base = r'X:\bl1-5\Alexis\2025\202505\20250501\OneD_integrated_WAXS_01\Reduction'

    for keywords in all_keywords:
        # Read SAXS data
        result_data_saxs, m_keys_saxs, legend_saxs = read_data_files(folder_path_saxs_base, keywords)

        # Read WAXS data
        result_data_waxs, m_keys_waxs, legend_waxs = read_data_files(folder_path_waxs_base, keywords)

        # Create SAXS-WAXS combined plot
        create_saxs_waxs_plot(
            folder_path_saxs_base,
            folder_path_waxs_base,
            keywords,
            m_keys_saxs,
            result_data_saxs,
            result_data_waxs,
            legend_saxs,
            legend_waxs,
        )
# https://pubs.rsc.org/en/content/articlelanding/2021/gc/d0gc03536j

# Data Reduction

## Dark

Updating this dark to be for May 11th - using darks within macro79_CapA_toluene_SBA-12nm_10wtpt_run3

In [ ]:


base_dir = r'/Users/alexisvoulgaropoulos/Library/CloudStorage/OneDrive-Stanford/BOTTLE/BEAMTIME/May2025_BL1-5_data/20250501/'

raw_dir = os.path.join(base_dir,"macro79_CapA_toluene_SBA-12nm_10wtpt_run3")

#raw_dir_meth = r'X:\bl1-5\Anjani\RedesignedPlastic\Jan2025\Fibers3'


common_keyword = ""  # folder name
keywords = ["*dark*" ]# file name

#keep these as zero for the dark - what ANJANI says.....
i0_offset = 0
bstop_offset = 0 

# loop over the keywords
for keyword in keywords:
     keywords, SAXS_folder_name, WAXS_folder_name,data_folder = get_keywords(raw_dir, common_keyword, keyword)
     
     process_SAXS_data(SAXS_folder_name, keywords, data_folder, ai, saxs_mask, i0_offset, bstop_offset, i0_air=0, bstop_air=0)
     #process_WAXS_data(WAXS_folder_name, keywords, data_folder, ai_w, waxs_mask, i0_offset, bstop_offset, i0_air, bstop_air)


## M79 SBA-15 / Toluene run 2

### Dark for run 2

In [ ]:
base_dir = r'/Users/alexisvoulgaropoulos/Library/CloudStorage/OneDrive-Stanford/BOTTLE/BEAMTIME/May2025_BL1-5_data/20250501/'

raw_dir = os.path.join(base_dir, 'macro79_CapA_toluene_SBA-12nm_10wtpt_run3')

i0_offset = 0
bstop_offset = 0

common_keyword = ""  # folder name
keywords = ['*M79*dark*']# file name

result = get_keywords(raw_dir, common_keyword, keyword)


for keyword in keywords:
     keywords, SAXS_folder_name, WAXS_folder_name,data_folder = get_keywords(raw_dir, common_keyword, keyword)
     
     process_SAXS_data(SAXS_folder_name, keywords, data_folder, ai, saxs_mask, i0_offset, bstop_offset, i0_air=0, bstop_air=0)
     #process_WAXS_data(WAXS_folder_name, keywords, data_folder, ai_w, mask=None, i0_offset=0, bstop_offset=0, i0_air=0, bstop_air=0)
     print(f"Keywords: {keywords}, SAXS_folder_name: {SAXS_folder_name}, WAXS_folder_name: {WAXS_folder_name}, data_folder: {data_folder}")

### SBA15 X Toluene run 2

In [ ]:
import re

base_dir = r'/Users/alexisvoulgaropoulos/Library/CloudStorage/OneDrive-Stanford/BOTTLE/BEAMTIME/May2025_BL1-5_data/20250501/'

raw_dir = os.path.join(base_dir, 'macro79_CapA_toluene_SBA-12nm_10wtpt_run3')

i0_offset = (2.397306 + 2.214361) / 2
bstop_offset = (0.530458 + 0.544308) / 2

common_keyword = ""  # folder name
keywords = ['*M79*capA*']# file name



for keyword in keywords:
     keywords, SAXS_folder_name, WAXS_folder_name,data_folder = get_keywords(raw_dir, common_keyword, keyword)

     # Function to extract the number after 'ctr'
     #def extract_ctr_value(s):
          #match = re.search(r'ctr(\d+)', s)
          #return int(match.group(1)) if match else -1  # fallback to -1 if no match

     # Sort the keywords list based on the ctr number
     #keywords.sort(key=extract_ctr_value)

     keywords = keywords[4:]
     
     process_SAXS_data(SAXS_folder_name, keywords, data_folder, ai, saxs_mask, i0_offset, bstop_offset, i0_air=0, bstop_air=0)
     #process_WAXS_data(WAXS_folder_name, keywords, data_folder, ai_w, mask=None, i0_offset=0, bstop_offset=0, i0_air=0, bstop_air=0)




     #Excluding the first 10 items from the keywords list (since the experiment starts around ctr 9)
     #print(f"keywords are {keywords}")


     #process_SAXS_data(SAXS_folder_name, keywords, data_folder, ai, saxs_mask, i0_offset, bstop_offset, i0_air=0, bstop_air=0)
     

In [ ]:
raw_dir = os.path.join(base_dir, 'macro79_CapA_toluene_SBA-12nm_10wtpt_run3')

i0_offset = (2.397306 + 2.214361) / 2 #dark values, averaged
bstop_offset = (0.530458 + 0.544308) / 2 #dark values, averaged

common_keyword = ""  # folder name
keywords = ['*M79*capA*']# file name


for keyword in keywords:
     keywords, SAXS_folder_name, WAXS_folder_name,data_folder = get_keywords(raw_dir, common_keyword, keyword)

     # Function to extract the number after 'ctr'
     def extract_ctr_value(s):
          match = re.search(r'ctr(\d+)', s)
          return int(match.group(1)) if match else -1  # fallback to -1 if no match

     # Sort the keywords list based on the ctr number
     keywords.sort(key=extract_ctr_value)

     #keywords = keywords[1:]
     
     process_SAXS_data(SAXS_folder_name, keywords, data_folder, ai, saxs_mask, i0_offset, bstop_offset, i0_air=0, bstop_air=0)
     #process_WAXS_data(WAXS_folder_name, keywords, data_folder, ai_w, mask=None, i0_offset=0, bstop_offset=0, i0_air=0, bstop_air=0)




     #Excluding the first 10 items from the keywords list (since the experiment starts around ctr 9)
     #print(f"keywords are {keywords}")


     #process_SAXS_data(SAXS_folder_name, keywords, data_folder, ai, saxs_mask, i0_offset, bstop_offset, i0_air=0, bstop_air=0)
     

### Pure toluene, run 2

In [ ]:
import re

base_dir = r'/Users/alexisvoulgaropoulos/Library/CloudStorage/OneDrive-Stanford/BOTTLE/BEAMTIME/May2025_BL1-5_data/20250501/'

raw_dir = os.path.join(base_dir, 'macro77_20250511_CapA_toluene')

print(raw_dir)

common_keyword = ""  # folder name
keywords = ["*capA*" ]# file name


i0_offset = 2.397306 # intial dark values from run 2
bstop_offset = 0.530458 # intial dark values from run 2

# loop over the keywords
for keyword in keywords:
     keywords, SAXS_folder_name, WAXS_folder_name,data_folder = get_keywords(raw_dir, common_keyword, keyword)
     
     process_SAXS_data(SAXS_folder_name, keywords, data_folder, ai, saxs_mask, i0_offset, bstop_offset, i0_air=0, bstop_air=0)
     #process_WAXS_data(WAXS_folder_name, keywords, data_folder, ai_w, waxs_mask, i0_offset, bstop_offset, i0_air, bstop_air)





### M76 glass

In [ ]:
import re

base_dir = r'/Users/alexisvoulgaropoulos/Library/CloudStorage/OneDrive-Stanford/BOTTLE/BEAMTIME/May2025_BL1-5_data/20250501/'

raw_dir = os.path.join(base_dir, 'macro76_20250511_CapA_empty')

print(raw_dir)

common_keyword = ""  # folder name
keywords = ["*capA*" ]# file name


i0_offset = 2.397306 # intial dark values from run 2
bstop_offset = 0.530458 # intial dark values from run 2

# loop over the keywords
for keyword in keywords:
     keywords, SAXS_folder_name, WAXS_folder_name,data_folder = get_keywords(raw_dir, common_keyword, keyword)
     
     process_SAXS_data(SAXS_folder_name, keywords, data_folder, ai, saxs_mask, i0_offset, bstop_offset, i0_air=0, bstop_air=0)
     #process_WAXS_data(WAXS_folder_name, keywords, data_folder, ai_w, waxs_mask, i0_offset, bstop_offset, i0_air, bstop_air)





## M83 SBA-15 / Toluene run 3

### Dark for M83 - SBA 12 nm

In [ ]:
raw_dir = os.path.join(base_dir, 'macro83_CapA_toluene_SBA-12nm_10wtpt_run4')

i0_offset = 0
bstop_offset = 0

common_keyword = ""  # folder name
keywords = ['*M83*dark*']# file name

result = get_keywords(raw_dir, common_keyword, keyword)


for keyword in keywords:
     keywords, SAXS_folder_name, WAXS_folder_name,data_folder = get_keywords(raw_dir, common_keyword, keyword)
     
     process_SAXS_data(SAXS_folder_name, keywords, data_folder, ai, saxs_mask, i0_offset, bstop_offset, i0_air=0, bstop_air=0)
     #process_WAXS_data(WAXS_folder_name, keywords, data_folder, ai_w, mask=None, i0_offset=0, bstop_offset=0, i0_air=0, bstop_air=0)
     print(f"Keywords: {keywords}, SAXS_folder_name: {SAXS_folder_name}, WAXS_folder_name: {WAXS_folder_name}, data_folder: {data_folder}")

### SBA15 X Toluene run 3

In [ ]:
raw_dir = os.path.join(base_dir, 'macro83_CapA_toluene_SBA-12nm_10wtpt_run4')

i0_offset = (2.434937 + 2.276866) / 2
bstop_offset = (0.549984 + 0.55664) / 2

common_keyword = ""  # folder name
keywords = ['*M83*capA*']# file name


for keyword in keywords:
     keywords, SAXS_folder_name, WAXS_folder_name,data_folder = get_keywords(raw_dir, common_keyword, keyword)

     # Function to extract the number after 'ctr'
     def extract_ctr_value(s):
          match = re.search(r'ctr(\d+)', s)
          return int(match.group(1)) if match else -1  # fallback to -1 if no match

     # Sort the keywords list based on the ctr number
     keywords.sort(key=extract_ctr_value)

     keywords = keywords[1:35]
     
     process_SAXS_data(SAXS_folder_name, keywords, data_folder, ai, saxs_mask, i0_offset, bstop_offset, i0_air=0, bstop_air=0)
     #process_WAXS_data(WAXS_folder_name, keywords, data_folder, ai_w, mask=None, i0_offset=0, bstop_offset=0, i0_air=0, bstop_air=0)




     #Excluding the first 10 items from the keywords list (since the experiment starts around ctr 9)
     #print(f"keywords are {keywords}")


     #process_SAXS_data(SAXS_folder_name, keywords, data_folder, ai, saxs_mask, i0_offset, bstop_offset, i0_air=0, bstop_air=0)
     

### pure toluene, run 3

In [ ]:
import re

base_dir = r'/Users/alexisvoulgaropoulos/Library/CloudStorage/OneDrive-Stanford/BOTTLE/BEAMTIME/May2025_BL1-5_data/20250501/'

raw_dir = os.path.join(base_dir, 'macro82_20250512_CapA_toluene')

print(raw_dir)

common_keyword = ""  # folder name
keywords = ["*capA*" ]# file name


i0_offset = 2.434937 # intial dark values from run 2
bstop_offset = 0.549984 # intial dark values from run 2

# loop over the keywords
for keyword in keywords:
     keywords, SAXS_folder_name, WAXS_folder_name,data_folder = get_keywords(raw_dir, common_keyword, keyword)
     
     process_SAXS_data(SAXS_folder_name, keywords, data_folder, ai, saxs_mask, i0_offset, bstop_offset, i0_air=0, bstop_air=0)
     #process_WAXS_data(WAXS_folder_name, keywords, data_folder, ai_w, waxs_mask, i0_offset, bstop_offset, i0_air, bstop_air)





## M43 SBA15 12 nm run 4

### Dark for M43 - SBA 12 nm

In [ ]:
raw_dir = os.path.join(base_dir, 'macro36_20250504_capA_SBA12nm_capG_HDPE-SBA12nm_run3')

i0_offset = 0
bstop_offset = 0

common_keyword = ""  # folder name
keywords = ['*M36*dark*']# file name

result = get_keywords(raw_dir, common_keyword, keyword)


for keyword in keywords:
     keywords, SAXS_folder_name, WAXS_folder_name,data_folder = get_keywords(raw_dir, common_keyword, keyword)
     
     process_SAXS_data(SAXS_folder_name, keywords, data_folder, ai, saxs_mask, i0_offset, bstop_offset, i0_air=0, bstop_air=0)
     #process_WAXS_data(WAXS_folder_name, keywords, data_folder, ai_w, mask=None, i0_offset=0, bstop_offset=0, i0_air=0, bstop_air=0)
     print(f"Keywords: {keywords}, SAXS_folder_name: {SAXS_folder_name}, WAXS_folder_name: {WAXS_folder_name}, data_folder: {data_folder}")

### SBA15 X HDPE run 4

In [ ]:
raw_dir = os.path.join(base_dir, 'macro43_20250508_capD_HDPE_capH_HDPE-SBA12nm_run4')

i0_offset = (2.672296 + 2.025881) / 2
bstop_offset = (0.562345 + 0.553299) / 2

common_keyword = ""  # folder name
keywords = ['*M43*capH_HDPE*']# file name


for keyword in keywords:
     keywords, SAXS_folder_name, WAXS_folder_name,data_folder = get_keywords(raw_dir, common_keyword, keyword)

     # Function to extract the number after 'ctr'
     def extract_ctr_value(s):
          match = re.search(r'ctr(\d+)', s)
          return int(match.group(1)) if match else -1  # fallback to -1 if no match

     # Sort the keywords list based on the ctr number
     keywords.sort(key=extract_ctr_value)

     keywords = keywords[1:20]
     
     process_SAXS_data(SAXS_folder_name, keywords, data_folder, ai, saxs_mask, i0_offset, bstop_offset, i0_air=0, bstop_air=0)
     #process_WAXS_data(WAXS_folder_name, keywords, data_folder, ai_w, mask=None, i0_offset=0, bstop_offset=0, i0_air=0, bstop_air=0)




     #Excluding the first 10 items from the keywords list (since the experiment starts around ctr 9)
     #print(f"keywords are {keywords}")


     #process_SAXS_data(SAXS_folder_name, keywords, data_folder, ai, saxs_mask, i0_offset, bstop_offset, i0_air=0, bstop_air=0)
     

## M75 10 wt% SBA-15 in Toluene

### Dark for M75

In [ ]:
raw_dir = os.path.join(base_dir, 'macro75_CapA_toluene_SBA-12nm_10wtpt_run2')

i0_offset = 0
bstop_offset = 0

common_keyword = ""  # folder name
keywords = ['*M75*dark*']# file name

result = get_keywords(raw_dir, common_keyword, keyword)


for keyword in keywords:
     keywords, SAXS_folder_name, WAXS_folder_name,data_folder = get_keywords(raw_dir, common_keyword, keyword)
     
     process_SAXS_data(SAXS_folder_name, keywords, data_folder, ai, saxs_mask, i0_offset, bstop_offset, i0_air=0, bstop_air=0)
     #process_WAXS_data(WAXS_folder_name, keywords, data_folder, ai_w, mask=None, i0_offset=0, bstop_offset=0, i0_air=0, bstop_air=0)
     print(f"Keywords: {keywords}, SAXS_folder_name: {SAXS_folder_name}, WAXS_folder_name: {WAXS_folder_name}, data_folder: {data_folder}")

### 10 wt% SBA in Toluene run 2

In [ ]:
raw_dir = os.path.join(base_dir, 'macro75_CapA_toluene_SBA-12nm_10wtpt_run2')

i0_offset = (2.672296 + 2.025881) / 2
bstop_offset = (0.562345 + 0.553299) / 2

common_keyword = ""  # folder name
keywords = ['*M75*capA_SBA12*']# file name


for keyword in keywords:
     keywords, SAXS_folder_name, WAXS_folder_name,data_folder = get_keywords(raw_dir, common_keyword, keyword)

     # Function to extract the number after 'ctr'
     def extract_ctr_value(s):
          match = re.search(r'ctr(\d+)', s)
          return int(match.group(1)) if match else -1  # fallback to -1 if no match

     # Sort the keywords list based on the ctr number
     keywords.sort(key=extract_ctr_value)

     keywords = keywords[4:40]
     
     process_SAXS_data(SAXS_folder_name, keywords, data_folder, ai, saxs_mask, i0_offset, bstop_offset, i0_air=0, bstop_air=0)
     #process_WAXS_data(WAXS_folder_name, keywords, data_folder, ai_w, mask=None, i0_offset=0, bstop_offset=0, i0_air=0, bstop_air=0)




     #Excluding the first 10 items from the keywords list (since the experiment starts around ctr 9)
     #print(f"keywords are {keywords}")


     #process_SAXS_data(SAXS_folder_name, keywords, data_folder, ai, saxs_mask, i0_offset, bstop_offset, i0_air=0, bstop_air=0)
     